In [1]:
# Modules: gdal/3.6.4  
# Environment base: /g/data/xe2/John/geospatenv

In [2]:
# Note: For the canopy_height.download_new_tiles to work, you need to create an AWS account, then create a file named .aws/credentials in your home directory (e.g. /home/147/cb8590) with these contents:
# [default]
# aws_access_key_id = ACCESS_KEY
# aws_secret_access_key = SECRET_KEY

In [3]:
# Standard library
import os

# Local imports
os.chdir(os.path.join(os.path.expanduser('~'), "Projects/PaddockTS"))
from DAESIM_preprocess.util import gdata_dir, scratch_dir
from DAESIM_preprocess.terrain_tiles import terrain_tiles
from DAESIM_preprocess.slga_soils import slga_soils, asris_urls
from DAESIM_preprocess.ozwald_yearly import ozwald_yearly_average
from DAESIM_preprocess.ozwald_8day import ozwald_8day, ozwald_8day_abbreviations
from DAESIM_preprocess.ozwald_daily import ozwald_daily, ozwald_daily_abbreviations
from DAESIM_preprocess.silo_daily import silo_daily
from DAESIM_preprocess.canopy_height import canopy_height

In [4]:
# Choosing location
# lat, lon = -34.3890427, 148.469499
# buffer = 0.005  # 0.01 degrees is about 1km in each direction, so 2km total
# stub = "Test"
# start_year = 2020
# end_year = 2021

# Choosing location
lat, lon = -34.3890427, 148.469499
lat, lon = -35.200628, 149.145974
buffer=0.04
stub = "CRGU3"
start_year = 2020
end_year = 2021

# Specify output destinations
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = scratch_dir

In [5]:
%%time
# Download elevation from terrain tiles
terrain_tiles(lat, lon, buffer, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/CRGU3_terrain_original.tif
Saved /g/data/xe2/cb8590/Data/PadSeg/CRGU3_terrain.tif
CPU times: user 4.61 s, sys: 251 ms, total: 4.86 s
Wall time: 18.7 s


In [6]:
%%time
# The CSIRO soils API seems to frequently be randomly unavailable unfortunately
variables = ['Clay', 'Sand', 'Silt', 'pH_CaCl2']
slga_soils(variables, lat, lon, buffer, outdir, stub)

Downloaded /g/data/xe2/cb8590/Data/PadSeg/CRGU3_Clay_5-15cm.tif
Downloaded /g/data/xe2/cb8590/Data/PadSeg/CRGU3_Sand_5-15cm.tif
Failed to download Silt
Downloaded /g/data/xe2/cb8590/Data/PadSeg/CRGU3_Silt_5-15cm.tif
Downloaded /g/data/xe2/cb8590/Data/PadSeg/CRGU3_pH_CaCl2_5-15cm.tif
CPU times: user 230 ms, sys: 31.9 ms, total: 262 ms
Wall time: 1min 16s


In [7]:
%%time
# Download average climate data from SILO hosted on OzWald
variables = ["Tmax", "Tmin", "Pg"]
ozwald_yearly_average(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Saved: /g/data/xe2/cb8590/Data/PadSeg/CRGU3_Maximum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/CRGU3_Minimum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/CRGU3_Annual_Rainfall_2020_2021_average.tif
CPU times: user 207 ms, sys: 243 ms, total: 451 ms
Wall time: 1.65 s


In [8]:
%%time
# Download modelled variables from OzWald
variables = ['Ssoil']
ds = ozwald_8day(variables, lat, lon, buffer, start_year, end_year, outdir, stub)

Saved: /g/data/xe2/cb8590/Data/PadSeg/CRGU3_ozwald_8day.nc
CPU times: user 666 ms, sys: 165 ms, total: 831 ms
Wall time: 996 ms


In [9]:
%%time
# Download daily weather data hosted on OzWald
variables = ["VPeff", "Uavg"]
ds = ozwald_daily(variables, lat, lon, buffer, start_year, end_year, outdir, stub)

Saved: /g/data/xe2/cb8590/Data/PadSeg/CRGU3_ozwald_daily.nc
CPU times: user 279 ms, sys: 85.2 ms, total: 364 ms
Wall time: 438 ms


In [10]:
%%time
# Load data pre-downloaded to gdata from SILO's AWS server
ds_silo_daily = silo_daily(["daily_rain", "max_temp", "min_temp", "et_morton_actual", "et_morton_potential"], lat, lon, buffer, start_year, end_year, outdir, stub)
ds_silo_daily

Saved: /g/data/xe2/cb8590/Data/PadSeg/CRGU3_silo_daily.nc
CPU times: user 128 ms, sys: 1.05 s, total: 1.18 s
Wall time: 1.21 s


<xarray.Dataset>
Dimensions:              (lat: 1, lon: 1, time: 731)
Coordinates:
  * lat                  (lat) float64 -35.2
  * lon                  (lon) float64 149.2
  * time                 (time) datetime64[ns] 2020-01-01 ... 2021-12-31
Data variables:
    daily_rain           (time, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    crs                  (time) |S1 b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    max_temp             (time, lat, lon) float32 33.5 33.2 37.9 ... 28.9 30.7
    min_temp             (time, lat, lon) float32 13.8 11.5 15.5 ... 9.4 10.4
    et_morton_actual     (time, lat, lon) float32 0.9 3.1 4.5 ... 5.7 7.0 6.2
    et_morton_potential  (time, lat, lon) float32 11.0 9.4 11.3 ... 8.0 7.9 8.6
Attributes:
    department:               Department of Environment and Science
    department_short:         DES
    copyright:                Copyright - the State of Queensland Department ...
    site_url:                 http://www.longpaddock.qld.gov.au
    institution:              Queensland Government, Department of Environmen...
    raster_source:            Gridded surface was created by interpolating ob...
    raster_source_additions:  and other suppliers (see the SILO webpage for d...
    metadata_url:             http://qldspatial.information.qld.gov.au/catalo...
    reference:                Jeffrey, S.J., Carter, J.O., Moodie, K.B.A. and...
    disclaimer:               1. The user accepts all responsibility and risk...

In [11]:
%%time 
# Download and merge/crop canopy height tiles from their AWS server
canopy_height(lat, lon, buffer, outdir, stub, tmp_dir)

Saved: /g/data/xe2/cb8590/Data/PadSeg/CRGU3_canopy_height.tif
CPU times: user 28.1 s, sys: 981 ms, total: 29.1 s
Wall time: 29.2 s
